In [1]:
import aqt

from anki import scheduler_pb2

from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import scipy

from dateutil.parser import parse as dt_parse

import datetime, itertools, time

In [2]:
browser = aqt.dialogs._dialogs['Browser'][1]
table = browser.table
model = table._model

In [29]:
notes[0].__dict__

{'col': <anki.collection.Collection at 0x7f5e2c042540>,
 'id': 1659061121012,
 'guid': 'hME%?|Dw`;',
 'mid': 1539794822805,
 'mod': 1672608831,
 'usn': 2005,
 'tags': ['ann:technique', 'CET-05.05-TheSubstitutionRule'],
 'fields': ['CET-05.05-Xm01',
  '',
  '220727-2119: technique: substitution of square root of&nbsp; a function'],
 '_fmap': {'Front': (0,
   {'name': 'Front',
    'ord': 0,
    'sticky': False,
    'rtl': False,
    'font': 'Liberation Sans',
    'size': 20,
    'description': '',
    'plainText': False,
    'collapsed': False,
    'media': []}),
  'Back': (1,
   {'name': 'Back',
    'ord': 1,
    'sticky': False,
    'rtl': False,
    'font': 'Arial',
    'size': 20,
    'description': '',
    'plainText': False,
    'collapsed': False,
    'media': []}),
  'Annotations': (2,
   {'name': 'Annotations',
    'ord': 2,
    'sticky': False,
    'rtl': False,
    'font': 'Liberation Sans',
    'size': 20,
    'description': '',
    'plainText': False,
    'collapsed': False,

In [52]:
cards[0]

<Card: id=1672682213036, type=3, queue=1, due=1673462645, factor=2150, reps=6, lapses=1>

In [51]:
cards[1].note()

<Note: id=1672682252627, fields=['ECET-06.06-Xc029 (testing)', '', ''], tags=['ECET-06.06-ImproperIntegrals']>

In [50]:
from pylib.anki.cards import Card
from pylib.anki.notes import Note

def card_repr(self):
    return f"<Card: id={self.id}, type={self.type}, queue={self.queue}, due={self.due}, factor={self.factor}, reps={self.reps}, lapses={self.lapses}>"

def note_repr(self):
    return f"<Note: id={self.id}, fields={self.fields}, tags={self.tags}>"

cards[0].__class__.__repr__ = card_repr
notes[0].__class__.__repr__ = note_repr

In [26]:
CardAnswer = scheduler_pb2.CardAnswer

def build_answer(card, rating, seconds_taken, answered_at = None):
    if answered_at is None:
        answered_at = datetime.datetime.now()
    if type(answered_at) == datetime.datetime:
        answered_at = int(answered_at.timestamp() * 1000)
    ms_taken = int(seconds_taken * 1000)
    states = model.col._backend.get_scheduling_states(card.id)
    if rating == CardAnswer.AGAIN:
        new_state = states.again
    elif rating == CardAnswer.HARD:
        new_state = states.hard
    elif rating == CardAnswer.GOOD:
        new_state = states.good
    elif rating == CardAnswer.EASY:
        new_state = states.easy
    else:
        raise Exception("invalid rating")
        
    answer = CardAnswer(
        card_id=card.id,
        current_state=states.current,
        new_state=states.again,
        rating=rating,
        answered_at_millis=answered_at,
        milliseconds_taken=ms_taken,
    )
    return answer

def answer_card(answer):
    model.col._backend.answer_card_raw(answer.SerializeToString())

In [56]:
card = table.get_single_selected_card()
print(f"Note: {card.note()}")
print(f"Card: {card}")
print(f"Review IDs:\n{table.get_selected_review_ids()}")
answer_card(
    build_answer(
        answered_at = dt_parse('January 3, 2021 at 1711'),
        rating=CardAnswer.EASY,
        seconds_taken=10,
        card=table.get_single_selected_card(),
    )
)
card = table.get_single_selected_card()
print(f"*** After review:")
print(f"Card: {card}")
print(f"Review IDs:\n{table.get_selected_review_ids()}")

Note: <Note: id=1672682213036, fields=['ECET-06.06-Xc031 (testing)', '', '220803-1212: trouble: inverted a fraction during cancellation in ratio of two fractions.'], tags=['ECET-06.06-ImproperIntegrals']>
Card: <Card: id=1672682213036, type=3, queue=1, due=1673463428, factor=2150, reps=9, lapses=1>
Review IDs:
[1609718400000, 1609718700000, 1609719000000, 1609719060000, 1672682332155, 1672682347659, 1672682360378, 1673046179613, 1673397355817]
*** After review:
Card: <Card: id=1672682213036, type=3, queue=1, due=1673463566, factor=2150, reps=10, lapses=1>
Review IDs:
[1609718400000, 1609718700000, 1609719000000, 1609719060000, 1609719120000, 1672682332155, 1672682347659, 1672682360378, 1673046179613, 1673397355817]
